# Ask questions about how many linear ops there are

## Authors:
- **David W. Hogg** (NYU) (MPIA) (Flatiron)
- **Soledad Villar** (JHU)

## To-do items and bugs:
- Do something.

In [1]:
import itertools as it
import numpy as np
import geometric as geom
import finufft
import pylab as plt
%load_ext autoreload
%autoreload 2

In [2]:
D = 3
group_operators = geom.make_all_operators(D)
print(len(group_operators))

In [3]:
geom.test_group(group_operators)

group is closed under multiplication
group operators are the transposes of their inverses


True

In [4]:
geom.test_group_actions(group_operators)

passed (parity = -1) vector dot test.
passed (parity = -1) tensor times tensor test
passed (parity = -1) v T v test.
passed (parity = 1) vector dot test.
passed (parity = 1) tensor times tensor test
passed (parity = 1) v T v test.


True

In [ ]:
allfilters = {}
names = {}
maxn = {}
for M in [3, ]:
    maxn[(D, M)] = 0
    klist = (0, 1, 2)
    for k, parity in it.product(klist, (1, -1)):
        key = (D, M, k, parity)
        allfilters[key] = geom.get_unique_invariant_filters(M, k, parity, D, group_operators)
        n = len(allfilters[key])
        if n > maxn[(D, M)]:
            maxn[(D, M)] = n
        names[key] = ["{} {}".format(geom.ktensor.name(k, parity), i) for i in range(n)]

In [ ]:
dpi = 300
paritysign = {1: "+", -1: "-"}
for key in allfilters.keys():
    D, M, k, parity = key
    fig = geom.plot_filters(allfilters[key], names[key], maxn[(D, M)])
    plotname = "filter{}{}_{}_{}.png".format(paritysign[parity], k, D, M)

In [ ]:
# Make a sensible vector image on a 2-torus
N = 3
np.random.seed(42)
imagex = np.random.normal(size=(N, N))
imagey = np.random.normal(size=(N, N))
package = np.zeros((N, N, D))
filtered = False # True if you want the image to be "smooth".
if filtered:
    foo = np.pi * np.arange(-1. + 1. / N, 1., 2. / N)
    ys, xs = np.meshgrid(foo, foo) # ys, xs or xs, ys??
    ftx = finufft.nufft2d1(xs.flatten(), ys.flatten(), imagex.flatten().astype(complex), (12, 12))
    fty = finufft.nufft2d1(xs.flatten(), ys.flatten(), imagey.flatten().astype(complex), (12, 12))
    package[:, :, 0] = finufft.nufft2d2(xs.flatten(), ys.flatten(), ftx).reshape(N, N).real
    package[:, :, 1] = finufft.nufft2d2(xs.flatten(), ys.flatten(), fty).reshape(N, N).real
else:
    package[:, :, 0] = imagex
    package[:, :, 1] = imagey
package /= np.sqrt(np.mean(package ** 2))
print(package.shape)
vector_image = geom.geometric_image(package, 1, D).normalize()
print(vector_image)

In [ ]:
fig = geom.plot_image(vector_image)

In [ ]:
# How can we make a vector image from this vector image?
# 1. Convolve with scalar filters
M = 3
key = (2, M, 0, 1) # D M k parity
v_images = [vector_image.convolve_with(ff).normalize() for ff in allfilters[key]]
print(len(v_images))

In [ ]:
# 2. Convolve with pseudoscalar filters and Levi-Civita contract
# Oh wait, we don't have any pseudoscalars at 3x3!
key = (2, M, 0, -1) # D M k parity
v_images += [vector_image.convolve_with(ff).levi_civita_contract(0).normalize() for ff in allfilters[key]]
print(len(v_images))

In [ ]:
# 3. Convolve with 2-tensor filters and contract
key = (2, M, 2, 1) # D M k parity
v_images += [vector_image.convolve_with(ff).contract(0, 1).normalize() for ff in allfilters[key]]
v_images += [vector_image.convolve_with(ff).contract(0, 2).normalize() for ff in allfilters[key]]
print(len(v_images))

In [ ]:
# 4. Convolve with 2-pseudotensor filters, Levi-Civita contract, and contract
key = (2, M, 2, -1) # D M k parity
v_images += [vector_image.convolve_with(ff).levi_civita_contract(0).contract(0, 1).normalize() for ff in allfilters[key]]
v_images += [vector_image.convolve_with(ff).levi_civita_contract(0).contract(0, 2).normalize() for ff in allfilters[key]]
print(len(v_images))

In [ ]:
datablock = np.array([im.unpack().flatten() for im in v_images])
print(datablock.shape)
u, s, v = np.linalg.svd(datablock)
print("there are", np.sum(s > geom.TINY), "different images")